
<h1 align=center>Réseaux de neurones avec Keras</h1>

## Imports

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
from tensorflow import keras
import time

## Exercice 1 – Découverte de TensorFlow Playground

[TensorFlow Playground](http://playground.tensorflow.org).

- **Layers et patterns**: essayez d’entraîner le réseau de neurones par défaut en cliquant sur le bouton "Exécuter" (en haut à gauche). Remarquez comment il trouve rapidement une bonne solution pour la tâche de classification. Notez que les neurones de la première couche cachée ont appris des motifs simples, tandis que les neurones de la deuxième couche cachée ont appris à combiner les motifs simples de la première couche cachée en des motifs plus complexes. En général, plus il y a de couches, plus les motifs peuvent être complexes.

- **Fonction d'activation**: essayez de remplacer la fonction d'activation Tanh par la fonction d'activation ReLU, puis entraînez à nouveau le réseau. Notez qu'il trouve une solution encore plus rapidement, mais cette fois, les limites sont linéaires. Cela est dû à la forme de la fonction ReLU.

- **Minima locaux**: modifiez l’architecture du réseau pour n’avoir qu’une couche cachée avec trois neurones. Entraînez-le plusieurs fois (pour réinitialiser les poids du réseau, ajoutez et supprimez un neurone). Notez que le temps d’entraînement varie beaucoup et que parfois, il s’arrête sur un minimum local.

- **Trop petit**: supprimez maintenant un neurone pour ne garder que 2. Notez que le réseau de neurones est maintenant incapable de trouver une bonne solution, même si vous essayez plusieurs fois. Le modèle a trop peu de paramètres et sous-complète systématiquement l'ensemble de formation.

- **Assez grand**: ensuite, définissez un nombre de neurones à 8 et entraînez le réseau plusieurs fois. Notez qu'il est maintenant constamment rapide et ne reste jamais bloqué. Cela met en évidence une découverte importante de la théorie des réseaux de neurones: les grands réseaux de neurones ne se coincent presque jamais dans les minima locaux, et même lorsqu'ils le font, ces optima locaux sont presque aussi bons que l'optimum global. Cependant, ils peuvent rester bloqués sur de longs plateaux pendant longtemps.

- **Gradients profonds et nuls**: changez maintenant le jeu de données en spirale (jeu de données en bas à droite sous "DONNEES"). Changez l'architecture du réseau pour avoir 4 couches cachées de 8 neurones chacune. Notez que l'entraînement prend beaucoup plus de temps et reste souvent bloqué sur des plateaux pendant de longues périodes. Notez également que les neurones situés dans les couches les plus hautes (c’est-à-dire à droite) ont tendance à évoluer plus rapidement que les neurones situés dans les couches les plus basses (c.-à-d. À gauche). Ce problème, appelé le problème des "gradients disparus", peut être résolu en utilisant une meilleure initialisation du poids et d'autres techniques, de meilleurs optimiseurs (tels que AdaGrad ou Adam) ou en utilisant la normalisation par lots.

- **Ensuite**: Tester l'API pendant au moins une heure quand vous avez le temps !

## Exercice 2 – Classification d'images Fashion MNIST

Charger le jeu de données fashion MNIST. Keras a un certain nombre de fonctions (dont load_data()) pour charger des jeux de données populaires dans `keras.datasets`. Le jeu de données est déjà divisé en un jeu d'apprentissage et un jeu de tests, mais il peut être utile de fractionner davantage le jeu d'apprentissage pour obtenir un jeu de validation(5000 echantillons):

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

Afficher la taille du jeu d'entrainement :

In [ ]:
X_train.shape

Afficher la première ligne

In [ ]:
X_train[0]

Afficher l'echantilon en utilisant la fonction `imshow()` avec un color map en `'binary'`

In [ ]:
plt.imshow(X_train[0], cmap="binary")
plt.show()

Afficher les différentes valeurs des labels

In [ ]:
y_train

Noms des classes :

In [ ]:
class_names = ["T-shirt", "Pantalon", "Pull", "Robe", "Manteau",
               "Sandale", "Chemise", "Sneaker", "Sac", "Bottines"]

La première image correspond donc à un manteau :

In [ ]:
class_names[y_train[0]]

Afficher les 50 premières images (utiliser des subplots en adaptant la taille de la figure): [documentation](https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.subplot.html#matplotlib.pyplot.subplot)

In [ ]:
lignes = 5
cols = 10
#plt.figure(figsize=(cols*1.4, lignes * 1.8))

for l in range(lignes):
    for c in range(cols):
        index = cols * l + c
        plt.subplot(lignes, cols, index + 1)
        plt.imshow(X_train[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        #plt.title(class_names[y_train[index]]+"\n"+str(y_train[index]))
plt.show()

### Construction d'un réseau de neurones de classification avec Keras

### 2.1)
Construire un modèle séquentiel (keras.models.Sequential),(voir [doc](https://keras.io/models/about-keras-models/) ), sans aucun argument, puis ajoutez-y quatre couches en utilisant la méthode add ():
 - une couche Flatten (keras.layers.Flatten) pour convertir chaque image 28x28 en une rangée unique de 784 pixels. Comme il s'agit de la première couche du modèle, il faut spécifier l'argument input_shape en laissant de côté la taille du lot: [28, 28].
- une couche dense (keras.layers.Dense) avec 300 neurones (unités aka) et la fonction d'activation "relu".
- Une autre couche dense avec 100 neurones, également avec la fonction d'activation "relu".
- Une couche dense finale avec 10 neurones (un par classe) et la fonction d'activation "softmax" garantissant que la somme de toutes les probabilités de classe estimées pour chaque image est égale à 1.

In [ ]:
model = keras.models.Sequential()

In [ ]:
model.add(keras.layers.Flatten(input_shape=[28, 28]))

In [ ]:
model.add(keras.layers.Dense(300, activation="relu"))

In [ ]:
model.add(keras.layers.Dense(100, activation="relu"))

In [ ]:
model.add(keras.layers.Dense(10, activation="softmax"))

In [ ]:
#Autre manière d'instancier un reseau keras
"""
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
"""

### 2.2)
Utiliser la méthode `summary()` pour étudier l'output.

In [ ]:
model.summary()

### 2.3) Compilation
Une fois le modèle créé, il faut utiliser la méthode `compile ()` pour spécifier la fonction `loss` et l'` optimiseur` à utiliser. On utilisera "sparse_categorical_crossentropy" pour le loss et l'optimiseur "sgd" (descente de gradient stochastique), spécifier également `metrics` à `accuracy`

In [ ]:
model.compile(optimizer = 'sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

### 2.4) Entrainnement
Entrainer le modèle en utilisant la méthode `fit()` avec 10 epochs et en activant les données de validation<br/>
**Remarque**: la méthode `fit ()`retourne un objet `History`contenant des statistiques d'entraînement. Il peut être utile de conserver le résultat dans une variable : (hist = model.fit (...)).<br>
Afficher ensuite le contenu de hist.history

In [ ]:
hist= model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

In [ ]:
hist.history

### 2.5) Courbe de performances
Construire un dataframe avec les données de l'attribut *history* de l'objet *hist* puis en faire une représenatation graphique (méthode plot())<br>

In [ ]:
pd.DataFrame(hist.history).plot(figsize=(8,5))

### 2.6) Continuer un entrainnement
Relancer `model.fit()` , et remarquer qu'il reprend là où il s'était arrété.

In [ ]:
hist= model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

### 2.7) Evaluation
Evaluer le modèle sur le jeu de test avec la méthode `evaluate()`

In [ ]:
test_loss, test_accuracy= model.evaluate(X_test, y_test)
print(f'Test accuracy {test_accuracy}')
print(f'Test_loss {test_loss}')

### 2.8) Prédiction
Définir X par les 10 premières instances du jeu de tests. Utiliser la méthode de prédict() du modèle pour estimer la probabilité de chaque classe pour chaque instance (arrondir les résultats au 100ème):

In [ ]:
X=X_test[:10]
lignes = 1
cols = 10
#plt.figure(figsize=(cols*1.4, lignes * 1.8))

for l in range(lignes):
    for c in range(cols):
        index = cols * l + c
        plt.subplot(lignes, cols, index + 1)
        plt.imshow(X[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        #plt.title(class_names[y_train[index]]+"\n"+str(y_train[index]))
plt.show()

In [ ]:
y_prob = model.predict(X)
y_prob.round(3)

Utilisez `np.argmax ()`pour obtenir l'ID de classe de la classe la plus probable pour chaque instance. *Utiliser 'axis=1'*

In [ ]:
y_pred = y_prob.argmax(axis=1)
y_pred

## Exercice 3 – Normalisation

### 3.1)
Pour l'utilisation de la descente de gradient, il est généralement préférable de s'assurer que les entités soient mises à l'échelle, de préférence avec une distribution normale. Uniformiser les valeurs des pixels et tester les performances du réseau.

- Utiliser `StandardScaler`de Scikit-Learn.
- Refaire une instance du modèle puis relancer la méthode fit()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
X_valid_scaled = scaler.transform(X_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
X_test_scaled = scaler.transform(X_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)

In [ ]:
model_sc = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_sc.compile(optimizer = 'sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')
hist2 = model_sc.fit(X_train_scaled, y_train, epochs=15,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
test_loss, test_accuracy= model_sc.evaluate(X_test_scaled, y_test)
print(f'Test accuracy {test_accuracy}')
print(f'Test_loss {test_loss}')

### 3.2) Courbes de performances
Tracer les courbes de loss et d'accuracy et comparer avec celles obtenues dans la première partie.

In [ ]:
pd.DataFrame(hist2.history).plot(figsize=(8,5))

### 3.3) Prédiction
Refaire la prédiction de la question 2.8

In [ ]:
y_prob = model_sc.predict(X)
y_prob.round(3)

In [ ]:
y_pred = y_prob.argmax(axis=1)
y_pred

In [ ]:
n_rows = 1
n_cols = 10
plt.figure(figsize=(n_cols*1.4, n_rows * 1.8))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_test[index]]+"\n"+class_names[y_pred[index]])
plt.show()

## Exercice 4 – Les Callbacks

### 4.1)
La méthode `fit()` accepte un argument `callbacks`. Entrainer le modèle avec un grand nombre de `epoch` en y ajoutant les *callbacks* suvants (les callback proviennent de la librairie `keras.callbacks` et sont définis dans un tableau :
* `EarlyStopping`: avec `patience=5` pour arrêter l'apprentissage dès que le modèle ne change plus (5 epoch de marge)
* `ModelCheckpoint`: specifie le nom du modèle à enregistrer (exemple `"mnist_fashion.h5"`) avec `save_best_only=True` pour enregistrer le meilleur modèle obtenu.

callbacks=\[keras.callbacks.EarlyStopping......,
    keras.callbacks........\]<br/>
history = model.fit(............,
                    callbacks=callbacks)

Le callback `EarlyStopping` permettra de réduire les risques de sur-apprentissage.

In [ ]:
model_final = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model_final.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd", metrics=["accuracy"])

In [ ]:
callbacks = [
    
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint("modele_mnist_fashion.h5", save_best_only=True),
]

In [ ]:
hist3 = model_final.fit(X_train_scaled, y_train, epochs=50,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=callbacks)

### 4.2)

Grâce aux callbacks, le dernier modèle enregistré est le meilleur sur le jeu de validation.<br>
Charger le modèle avec 'keras. Models. load_model () et refaire des prédictions.

In [ ]:
model_reload = keras.models.load_model("modele_mnist_fashion.h5")

In [ ]:
model_reload.summary()

In [ ]:
y_prob = model_reload.predict(X)
y_prob.round(3)

In [ ]:
n_rows = 1
n_cols = 10
plt.figure(figsize=(n_cols*1.4, n_rows * 1.8))
for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(X[index], cmap="binary", interpolation="nearest")
        plt.axis('off')
        plt.title(class_names[y_test[index]]+"\n"+class_names[y_pred[index]])
plt.show()

Une liste de callbacks est disponible sur https://keras.io/callbacks/

## 5. Supervision avec tensorboard

In [ ]:
# Chargement de l'extension Tensorboard pour notebook
%load_ext tensorboard

In [ ]:
#Definition du dossier des logs d'événements
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
#Fonction de lancement du serveur tensorboard
def tb(logdir=log_dir, port=6006, open_tab=True, sleep=2):
    import subprocess
    proc = subprocess.Popen(
        "tensorboard --logdir={0} --port={1}".format(logdir, port))
    if open_tab:
        import time
        time.sleep(sleep)
        import webbrowser
        webbrowser.open("http://127.0.0.1:{}/".format(port))
    return proc

In [ ]:
#Redéfinition du modèle et des callbacks
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd", metrics=["accuracy"])


callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    keras.callbacks.EarlyStopping(patience=5),
    keras.callbacks.ModelCheckpoint("my_mnist_model.h5", save_best_only=True),
]
history = model.fit(X_train_scaled, y_train, epochs=10,
                    validation_data=(X_valid_scaled, y_valid),
                    callbacks=callbacks)

In [ ]:
#Lancement du serveur tensorboard
server =tb()

In [ ]:
#Arrêt du serveur
server.kill()